In [1]:

import numpy as np 
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.table import Table
from astropy.table import vstack
from astropy.table import Column
import os
from scipy import optimize
from astroML.linear_model import TLS_logL
rcParams['ytick.labelsize'] = 15
rcParams['xtick.labelsize'] = 15
rcParams['axes.labelsize'] = 20
rcParams['axes.linewidth'] = 2
rcParams['font.size'] = 15
rcParams['axes.titlesize'] = 18


# Read in Chelsea's data ...
# Use the result of reading her data band by band,  
# keeping only those where:
# Npts  > 10 , Plike - Pnoise  > 2 , Plike - Pinf > 0.05 ,  edge == 0 

# calculating lambdaRF = lambdaOBS / (1+redshift)
# calculating SFinf  = np.power(10, log10sigmahat) * sqrt(np.power(10, log10tau / 365.)  
# calculating tauRF = np.power(10,log10tau  / (1.+ redshift)
# calculating 4000 A values : 
#   SFinf4000 =  SFinf  * np.power(4000 / lambdaRF , -0.479)
#   tau4000 =  tauRF * np.power(4000 /  lambdaRF  , 0.17 )
# cross-matching to Shen2011, with DR7 data,
# adding to each band updated M_BH 

# a dictionary of five filters .... 
s82 = np.load('s82drw_shen2008_dic.npy').item()



In [3]:
s82['u']['M_i_1','M_i_2', 'M_i_corr']

M_i_1,M_i_2,M_i_corr
float64,float64,float64
-23.901,-23.901,-24.181
-23.399,-23.399,-23.576
-24.735,-24.735,-25.058
-23.121,-23.121,-24.045
-26.451,-26.451,-26.974
-24.917,-24.917,-25.288
-27.263,-27.263,-27.856
-24.427,-24.427,-24.898
-25.251,-25.251,-25.695


In [16]:
s82['u']

SDR5ID,ra,dec,redshift,M_i_1,mass_BH,chi2_pdf,log10tau,log10sigmahat,log10tau_lim_lo,log10tau_lim_hi,log10sigmahat_lim_lo,log10sigmahat_lim_hi,edge,Plike,Pnoise,Pinf,mu,Npts,lambdaRF,SFinf,tauRF,SFinf4000,tau4000,d2d_asec_dbqso,M_i_2,M_i_corr
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64
301,2.169302,1.238649,1.073,-23.901,0.0,0.968284,1.9955,-0.723982,1.53265,3.72459,-1.23257,-0.584464,0,94.7333,90.6945,92.8552,20.7869,60,1698.0221900627112,0.09831540391013008,47.742009834496045,0.0652195355955333,55.228153722153294,0.0,-23.901,-24.181
144,1.091028,0.962126,0.7867,-23.399,0.0,1.14117,2.27673,-0.325921,2.08917,4.02532,-0.510504,-0.261182,0,63.9319,42.5078,62.2803,20.7824,47,1970.1124979011586,0.3398580551150435,105.84695349396134,0.2420873148515182,119.38921225751962,0.0,-23.399,-23.576
58,0.331289,0.17723,1.6199,-24.735,0.0,1.23288,1.87484,-0.31862,1.62701,3.0311,-0.636571,-0.236789,0,72.3127,61.8195,69.3966,20.5453,53,1343.5627314019619,0.21759733283957713,28.61246570406551,0.12903347377913088,34.44305240052322,0.0,-24.735,-25.058
190,1.364696,-0.098956,0.6125,-23.121,0.0,1.0239,2.47534,-0.394771,2.32222,4.21751,-0.522501,-0.354452,0,99.8258,56.6434,98.5452,19.8859,57,2182.9457364341083,0.364546193475551,185.28500633684095,0.2727516586369636,205.3776477218901,0.0,-23.121,-24.045
36,0.221552,-0.292485,2.7563,-26.451,9.361,1.10691,1.70714,-0.521248,1.28728,3.6222,-1.09609,-0.386069,0,81.792,78.9905,79.9715,20.6848,55,937.0923515161196,0.11250596983818298,13.563748539736254,0.05614002455760786,17.359088665217733,0.0,-26.451,-26.974
49,0.28137,0.686984,1.3988,-24.917,8.817,0.748652,2.62304,-0.641666,2.48696,4.32531,-0.750577,-0.597245,0,124.681,63.373,123.61,20.0128,59,1467.4003668500916,0.24474147199702426,175.00318784502468,0.15139015111738424,207.530958735004,0.0,-24.917,-25.288
370,2.607954,0.913245,2.8473,-27.263,0.0,1.00762,3.23382,-0.797514,2.89732,4.5885,-0.964445,-0.654236,0,66.2818,52.3143,66.1312,21.2105,54,914.9273516492085,0.34534213353974985,445.3115376609138,0.17035975961027092,572.2402768592847,0.0,-27.263,-27.856
229,1.669492,0.643952,1.3383,-24.427,8.666,0.940847,2.22321,-0.350026,2.01398,3.51335,-0.492537,-0.289666,0,88.2909,55.0468,85.9202,20.7157,61,1505.3671470726595,0.30229611635955245,71.50061385002057,0.18929387327460825,84.42299498899229,0.0,-24.427,-24.898
699,4.850273,-0.276395,1.769,-25.251,0.0,1.30656,2.58011,-0.69975,2.38881,4.36352,-0.9316,-0.60632,0,80.538,67.5241,79.686,20.1582,49,1271.2170458649332,0.20377859697231657,137.33683804432746,0.11767740204237148,166.88599502342325,0.0,-25.251,-25.695


In [28]:

fit_AC = False
fit_ACD = True
# make identical declarations of variables 
# as in Chelsea's code ..
A = [] ; C = [] ; D = []
#f = 'u'
for f in 'ugriz':
    var = s82[f]['SFinf'].data ** 2.0  # variance in mag^2, equal to sigmahat^2*tau/365 = SF_inf^2 
    tau = s82[f]['log10tau'].data # where tau is best-fit timescale in days
    lRF = s82[f]['lambdaRF'].data # rest-frame wavelength in Angstroms
    M_i_corr =  s82[f]['M_i_1'] #+ 1.196 # s82[f]['M_i_1'] # 
    # ['M_i_corr'].data # Absolute Magnitude M_i, K-corrected to z=0
    mass_bh = s82[f]['mass_BH'].data #  BH mass in log solar masses
    redshift = s82[f]['redshift'].data


    select = mass_bh > 0 
    print('Of ', len(select), ' quasars in that band, ',select.sum() , ' have M_BH')
    f = np.log10(np.sqrt(var))[select]  # SF_inf array
    x = np.log10(lRF/4000.)[select]     # Rest-wavelength array
    y = (M_i_corr+23.)[select]          # Absolute Magnitude M_i, K-corrected to z=0
    z = (mass_bh-9.)[select]            # BH mass in 10^9 solar masses
    red = np.log10(1.+redshift)[select]  #  redshift array; not needed


    #lum = y
    #mbh = z 


    # First, remove wavelength trend: 
    lampwrsf = - 0.479
    fo = f - lampwrsf*x 

    # Setup errors 
    #sigma_lum = 1e-5 #(set arbitrarily small) ; x1
    #sigma_mbh = 0.2 # BH mass errors in dex   ; x2
    #sigma_z = 1e-5  # (set arbitrarily small) ; NOT USED 
    sigma_y = 0.01  # uncertainty in SFinf   ; y1 

    y1 = fo  # SFinf 
    x1 = y  # luminosity 
    x2 = z # BH mass 


    # do linear regression
    from sklearn import linear_model
    regr = linear_model.LinearRegression()

    # fitting  A,C (setting D, E == 0 )
    if fit_AC : 
    # fourth line 
        X = x1.reshape(-1,1)
    
    # fitting A,C,D (setting E == 0 , fifth line)
    if fit_ACD : 
        X = np.reshape([x1,x2], [-1,2])
        
    print(np.shape(X))
    print(np.shape(y1))
    dy =  np.ones_like(y1) * sigma_y
    print(np.shape(dy))
    regr.fit(X, y1, dy )

    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)

    print('A=', regr.intercept_)
    print('C=', regr.coef_[0])  # M_i coeff
    if fit_ACD : 
        print('D=', regr.coef_[1])  # M_BH coeff 
    A.append(regr.intercept_)
    C.append(regr.coef_[0])
    if fit_ACD : 
        D.append(regr.coef_[1])
print('Band-averaged A=', np.mean(A))
print('Band-averaged C=',np.mean(C))
if fit_ACD : 
    print('Band-averaged D=',np.mean(D))

Of  7014  quasars in that band,  4660  have M_BH
(4660, 2)
(4660,)
(4660,)
Intercept: 
 -0.7992920811741402
Coefficients: 
 [-0.00657789  0.00577838]
A= -0.7992920811741402
C= -0.006577887396761778
D= 0.005778380692746397
Of  7408  quasars in that band,  4702  have M_BH
(4702, 2)
(4702,)
(4702,)
Intercept: 
 -0.7871162686110268
Coefficients: 
 [ 0.00290088 -0.00207115]
A= -0.7871162686110268
C= 0.0029008826281390772
D= -0.0020711497472982363
Of  6871  quasars in that band,  4352  have M_BH
(4352, 2)
(4352,)
(4352,)
Intercept: 
 -0.7969449136937193
Coefficients: 
 [ 0.00551432 -0.0034194 ]
A= -0.7969449136937193
C= 0.00551432118260715
D= -0.0034194015079403923
Of  6814  quasars in that band,  4338  have M_BH
(4338, 2)
(4338,)
(4338,)
Intercept: 
 -0.8076331493407592
Coefficients: 
 [ 0.00545018 -0.00508578]
A= -0.8076331493407592
C= 0.005450184416007278
D= -0.005085780644430206
Of  5111  quasars in that band,  3500  have M_BH
(3500, 2)
(3500,)
(3500,)
Intercept: 
 -0.7830233800832171
Co

In [9]:
np.reshape([x1,x2], [-1,2])

array([[-3.378, -1.692],
       [-1.302, -3.002],
       [-2.143, -1.056],
       ...,
       [-0.119, -0.342],
       [ 0.1  , -0.088],
       [-0.324, -0.132]])